In [10]:
# !python3 -m pip --version # 查看pip版本
# !python -m pip install --upgrade pip #更新pip版本
# !python -m pip install paddlepaddle==2.0.1 -i https://mirror.baidu.com/pypi/simple

# [三岁白话系列]PaddlePaddle2.0——手写数字识别
三岁白话系列第6话

### 更新说明
原文档，数据集可以直接使用在最近的更新中需要使用ToTensor()进行转换，特此说明！
2021.1.18

### 参考文档：
Paddle官网：[https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/tutorial/quick_start/getting_started/getting_started.html#id3](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc1/tutorial/quick_start/getting_started/getting_started.html#id3)

paddle API查看地址：[https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api/paddle/vision/datasets/mnist/MNIST_cn.html](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api/paddle/vision/datasets/mnist/MNIST_cn.html)

MNIST书写数字识别官网：[http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/)

### CSDN地址
三岁白话系列CSDN：[https://blog.csdn.net/weixin_45623093/category_10616602.html](https://blog.csdn.net/weixin_45623093/category_10616602.html)

paddlepaddle社区号：[https://blog.csdn.net/PaddlePaddle](https://blog.csdn.net/PaddlePaddle)

In [11]:
# 导入paddle，查看版本型号
import paddle
from paddle.vision.transforms import ToTensor
print(paddle.__version__)

2.0.1


## 手写数字识别数据集
MNIST是手写识别数据集实例

其中训练集为60,000个示例，而测试集为10,000个示例



## 把训练集转换成图片
我们可以查看数据集里面的数据来进一步了解情况。

通过数据的下载和解压在对数据进行处理就得到了我们的`./mnist_train`文件夹

里面解压了0-9的文件可以进行查看

![](https://ai-studio-static-online.cdn.bcebos.com/abba1e6bcf7347f5ab533b833e92878e3c335f4c8ca448e696669d87d5343d55)

图像为3维，标签为1维。

图像大小为：28pix*28pix

In [12]:
# 导入第三方库
import numpy as np
import struct
from PIL import Image
import os

# 分别对train-images.idx3-ubyte和train-labels.idx1-ubyte进行处理最后得到png的图片格式

data_file = '/content/drive/MyDrive/AI Studio/data/train-images.idx3-ubyte'
# It's 47040016B, but we should set to 47040000B
data_file_size = 47040016
data_file_size = str(data_file_size - 16) + 'B'
 
data_buf = open(data_file, 'rb').read()
 
magic, numImages, numRows, numColumns = struct.unpack_from(
    '>IIII', data_buf, 0)
datas = struct.unpack_from(
    '>' + data_file_size, data_buf, struct.calcsize('>IIII'))
datas = np.array(datas).astype(np.uint8).reshape(
    numImages, 1, numRows, numColumns)
 
label_file = '/content/drive/MyDrive/AI Studio/data/train-labels.idx1-ubyte'
 
# It's 60008B, but we should set to 60000B
label_file_size = 60008
label_file_size = str(label_file_size - 8) + 'B'
 
label_buf = open(label_file, 'rb').read()
 
magic, numLabels = struct.unpack_from('>II', label_buf, 0)
labels = struct.unpack_from(
    '>' + label_file_size, label_buf, struct.calcsize('>II'))
labels = np.array(labels).astype(np.int64)
 
datas_root = 'mnist_train'
if not os.path.exists(datas_root):
    os.mkdir(datas_root)
 
for i in range(10):
    file_name = datas_root + os.sep + str(i)
    if not os.path.exists(file_name):
        os.mkdir(file_name)
 
for ii in range(numLabels):
    img = Image.fromarray(datas[ii, 0, 0:28, 0:28])
    label = labels[ii]
    file_name = datas_root + os.sep + str(label) + os.sep + \
        'mnist_train_' + str(ii) + '.png'
    img.save(file_name)

## 查看手写图片示例!
![](https://ai-studio-static-online.cdn.bcebos.com/754a3d4b606948ffada02dbe9df9d8cc972708a4380244acb5be41be94462ddd)
![](https://ai-studio-static-online.cdn.bcebos.com/c8acf750a75c4ae7b178f7ee8026b15f397ed0d8def7456e9e40b2fa5100e57e)
![](https://ai-studio-static-online.cdn.bcebos.com/49967c4230734fbebeb07f59f4fc0ef39790f2470b8348d79c467edb5c83c331)
![](https://ai-studio-static-online.cdn.bcebos.com/641f684be63d4756ae838671ee8bb7969547df82747048aab6aaa1e8ebbc12e2)
![](https://ai-studio-static-online.cdn.bcebos.com/0001fd9c10c6444d9add82b75ebac13c4bed5a17e84b4887a91ab55e1f5eacc5)
![](https://ai-studio-static-online.cdn.bcebos.com/ed0a0755db9b4e88b5aba1d34ac3e51c80a820c22be64848872b290ed4851b7e)
![](https://ai-studio-static-online.cdn.bcebos.com/8a72d2111f094fe0920f5e17e500bd4384fc1de237414eb5a3df5f04542101ae)
![](https://ai-studio-static-online.cdn.bcebos.com/8a052984d5494bd3a6b521a1997a1e8e27d35493fba644ec8c51478f02a21368)
![](https://ai-studio-static-online.cdn.bcebos.com/94cd1654112b47458f2025c12fc5d81d5691476de1e243139c7d856d2abafe8e)
![](https://ai-studio-static-online.cdn.bcebos.com/3e205743334c4758a926911adcd74c14d1cd1569e0f546979dd2b0165746e4bd)

## PaddlePaddle2.0 手写数据识别API
`paddle.vision.datasets.MNIST`

通过特定的参数获得对应的数据

image_path (str) - 图像文件路径，如果 download 设置为 True ，此参数可以设置为None。默认值为None。

label_path (str) - 标签文件路径，如果 download 设置为 True ，此参数可以设置为None。默认值为None。

chw_format (bool) - 若为 True 输出形状为[1, 28, 28], 否则为 [1, 784]。默认值为 True 。

mode (str) - 'train' 或 'test' 模式，默认为 'train' 。

download (bool) - 是否自定下载数据集文件。默认为 True 。

In [13]:
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=ToTensor())
val_dataset =  paddle.vision.datasets.MNIST(mode='test', transform=ToTensor())

Cache file /root/.cache/paddle/dataset/mnist/train-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /root/.cache/paddle/dataset/mnist/train-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/train-labels-idx1-ubyte.gz 
Begin to download
........
Download finished
Cache file /root/.cache/paddle/dataset/mnist/t10k-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /root/.cache/paddle/dataset/mnist/t10k-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/mnist/t10k-labels-idx1-ubyte.gz 
Begin to download
..
Download finished


## 模型搭建
通过`paddle.nn.Sequential`进行模型的搭建,把需要的数据都传进去。

### paddle.nn.Sequential(*layers)
顺序容器。子Layer将按构造函数参数的顺序添加到此容器中。传递给构造函数的参数可以Layers或可迭代的name Layer元组。

layers (tuple) - Layers或可迭代的name Layer对。

### paddle.nn.Linear(in_features, out_features, weight_attr=None, bias_attr=None, name=None)
线性变换层 。对于每个输入Tensor X ，计算公式为：`Out=XW+b`

其中， W 和 b 分别为权重和偏置。

in_features (int) – 线性变换层输入单元的数目。

out_features (int) – 线性变换层输出单元的数目。

weight_attr (ParamAttr, 可选) – 指定权重参数的属性。默认值为None，表示使用默认的权重参数属性，将权重参数初始化为0。具体用法请参见 ParamAttr 。

bias_attr (ParamAttr|bool, 可选) – 指定偏置参数的属性。 `bias_attr `为bool类型且设置为False时，表示不会为该层添加偏置。 `bias_attr` 如果设置为True或者None，则表示使用默认的偏置参数属性，将偏置参数初始化为0。具体用法请参见 [ParamAttr](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api/paddle/fluid/param_attr/ParamAttr_cn.html#cn-api-fluid-paramattr) 。默认值为None。

name (str，可选) – 具体用法请参见 [Name](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api_guides/low_level/program.html#api-guide-name) ，一般无需设置，默认值为None。

### class paddle.nn.ReLU(name=None)
ReLU激活层（Rectified Linear Unit）。计算公式如下：`ReLU(x)=max(0,x)`
	
其中，x 为输入的 Tensor

**形状:**

input: 任意形状的Tensor。

output: 和input具有相同形状的Tensor。

### paddle.nn.Dropout(p=0.5, axis=None, mode="upscale_in_train”, name=None)
Dropout是一种正则化手段，该算子根据给定的丢弃概率 p ，在训练过程中随机将一些神经元输出设置为0，通过阻止神经元节点间的相关性来减少过拟合。

p (float): 将输入节点置为0的概率， 即丢弃概率。默认: 0.5。

axis (int|list): 指定对输入 Tensor 进行Dropout操作的轴。默认: None。

mode (str): 丢弃单元的方式，有两种'upscale_in_train'和'downscale_in_infer'，默认: 'upscale_in_train'。

In [14]:
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(),
    paddle.nn.Linear(784, 512),  # 输入线性变换层数目为784个，输出为512个
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),  # 丢弃概率为0.2
    paddle.nn.Linear(512, 10)  # 输入线性变换层数目为512个，输出为10个
)


## 启用fit接口来开启我们的模型训练

### paddle.Model()
`Model` 对象是一个具备训练、测试、推理的神经网络。该对象同时支持静态图和动态图模式，通过 `paddle.disable_static()` 来切换。需要注意的是，该开关需要在实例化 `Model` 对象之前使用。输入需要使用 `paddle.static.InputSpec` 来定义。

### paddle.optimizer.Adam(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, parameters=None, weight_decay=None, grad_clip=None, name=None, lazy_mode=False)

能够利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率。

[具体参考](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api/paddle/optimizer/adam/Adam_cn.html)

### paddle.nn.loss.CrossEntropyLoss(weight=None, ignore_index=- 100, reduction='mean')
该OP计算输入input和标签label间的交叉熵损失 ，它结合了 LogSoftmax 和 NLLLoss 的OP计算，可用于训练一个 n 类分类器。

[具体参考](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api/paddle/nn/layer/loss/CrossEntropyLoss_cn.html)
### paddle.metric.Accuracy
计算准确率(accuracy)

topk (int|tuple(int)) - 计算准确率的top个数，默认是1。

name (str, optional) - metric实例的名字，默认是'acc'。

In [15]:
# 预计模型结构生成模型实例，便于进行后续的配置、训练和验证
model = paddle.Model(mnist)

# 模型训练相关配置，准备损失计算方法，优化器和精度计算方法
model.prepare(paddle.optimizer.Adam(parameters=mnist.parameters()),
              paddle.nn.CrossEntropyLoss(),
              paddle.metric.Accuracy())

# 开始模型训练
model.fit(train_dataset,
          epochs=5,
          batch_size=64,
          verbose=1)


The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 938/938 [==============================] - loss: 0.0925 - acc: 0.9300 - 27ms/step          
Epoch 2/5
step 938/938 [==============================] - loss: 0.0789 - acc: 0.9677 - 28ms/step          
Epoch 3/5
step 938/938 [==============================] - loss: 0.0413 - acc: 0.9780 - 27ms/step          
Epoch 4/5
step 938/938 [==============================] - loss: 0.0087 - acc: 0.9831 - 28ms/step          
Epoch 5/5
step 938/938 [==============================] - loss: 0.0784 - acc: 0.9862 - 27ms/step          


### 测试结果
通过loss和acc的结果来评断模型训练的质量和效率

In [16]:
model.evaluate(val_dataset, verbose=0)

{'acc': 0.9805, 'loss': [9.536748e-07]}

今天的白话就到这里了，我们下次再见啦！！！

这里是三岁，小白三岁，传说中的Paddle最菜程序员[狗头]